In [17]:
import leafmap.maplibregl as leafmap
import geopandas as gpd

In [14]:
# TODO: Import required libraries
import leafmap 
import shapely
from src.data_acquisition import fetch_pdok_buildings 
import geopandas as gpd

buildings2 = gpd.read_file('buildings.geojson')

buildings2.columns


Index(['fid', 'b3_h_50p', 'b3_h_70p', 'b3_h_max', 'b3_h_min', 'b3_dd_id',
       'identificatie', 'b3_pand_deel_id', 'b3_bag_bag_overlap',
       'b3_n_vlakken', 'b3_pw_onvoldoende', 'b3_t_run', 'b3_h_nok', 'b3_n_nok',
       'b3_extrusie', 'b3_is_glas_dak', 'b3_dak_type', 'b3_h_maaiveld',
       'b3_kas_warenhuis', 'b3_mutatie_ahn3_ahn4', 'b3_mutatie_ahn4_ahn5',
       'b3_nodata_fractie_ahn3', 'b3_nodata_fractie_ahn4',
       'b3_nodata_fractie_ahn5', 'b3_nodata_radius_ahn3',
       'b3_nodata_radius_ahn4', 'b3_nodata_radius_ahn5', 'b3_opp_buitenmuur',
       'b3_opp_dak_plat', 'b3_opp_dak_schuin', 'b3_opp_grond',
       'b3_opp_scheidingsmuur', 'b3_puntdichtheid_ahn3',
       'b3_puntdichtheid_ahn4', 'b3_puntdichtheid_ahn5', 'b3_pw_bron',
       'b3_pw_datum', 'b3_pw_selectie_reden', 'b3_rmse_lod12', 'b3_rmse_lod13',
       'b3_rmse_lod22', 'b3_val3dity_lod12', 'b3_val3dity_lod13',
       'b3_val3dity_lod22', 'b3_volume_lod12', 'b3_volume_lod13',
       'b3_volume_lod22', 'b3_bouwla

In [41]:
!pip install pydeck

In [48]:
# Pydeck demo: render 3D building extrusions using a small sample or fallback files
try:
    import pydeck as pdk
except Exception as err:
    raise ImportError("pydeck is required for this demo. Install with `pip install pydeck`.") from err

import json
from pathlib import Path
import webbrowser
import math
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime

# Helper: sanitize a GeoDataFrame before .to_json() to avoid Timestamp / numpy types
def sanitize_gdf(gdf: gpd.GeoDataFrame):
    g = gdf.copy()
    # convert datetime columns to ISO strings
    for col in g.columns:
        try:
            if pd.api.types.is_datetime64_any_dtype(g[col]) or pd.api.types.is_datetime64tz_dtype(g[col]):
                g[col] = g[col].astype(str)
        except Exception:
            # ignore columns that raise on dtype checks
            pass
    # convert numpy scalar columns to Python scalars
    for col in g.columns:
        if np.issubdtype(g[col].dtype, np.integer) or np.issubdtype(g[col].dtype, np.floating) or np.issubdtype(g[col].dtype, np.bool_):
            g[col] = g[col].apply(lambda x: None if (pd.isna(x)) else (x.item() if isinstance(x, np.generic) else x))
    return g

# Attempt to load a GeoJSON FeatureCollection from in-memory variables or disk
def load_geojson():
    # preferred in-memory sample created earlier in the notebook
    if 'sample_geojson' in globals():
        return sample_geojson
    # fallback: full JSON file produced by the MapLibre cell
    p = Path('buildings_3d_full_data.json')
    if p.exists():
        return json.loads(p.read_text(encoding='utf-8'))
    # fallback: geopandas objects from previous cells
    if 'b2' in globals():
        try:
            return json.loads(sanitize_gdf(b2).to_json())
        except Exception:
            # final fallback: convert datetimes to strings manually
            g = b2.copy()
            for col in g.columns:
                try:
                    if pd.api.types.is_datetime64_any_dtype(g[col]) or pd.api.types.is_datetime64tz_dtype(g[col]):
                        g[col] = g[col].astype(str)
                except Exception:
                    pass
            return json.loads(g.to_json())
    if 'buildings2' in globals():
        try:
            return json.loads(sanitize_gdf(buildings2).to_json())
        except Exception:
            g = buildings2.copy()
            for col in g.columns:
                try:
                    if pd.api.types.is_datetime64_any_dtype(g[col]) or pd.api.types.is_datetime64tz_dtype(g[col]):
                        g[col] = g[col].astype(str)
                except Exception:
                    pass
            return json.loads(g.to_json())
    # last fallback: raw geojson file
    p2 = Path('buildings.geojson')
    if p2.exists():
        return json.loads(gpd.read_file(p2).to_json())
    raise FileNotFoundError('No source geojson found (checked sample_geojson, buildings_3d_full_data.json, b2, buildings2, buildings.geojson)')

# sanitize properties in a FeatureCollection so every property is JSON-serializable
def sanitize_props_in_geojson(fc: dict):
    feats = fc.get('features', [])
    for feat in feats:
        props = feat.setdefault('properties', {})
        for k, v in list(props.items()):
            # numpy scalars -> python scalars
            if isinstance(v, np.generic):
                props[k] = v.item()
                continue
            # datetimes / pandas Timestamps -> ISO strings
            if isinstance(v, (datetime.date, datetime.datetime)):
                try:
                    props[k] = v.isoformat()
                except Exception:
                    props[k] = str(v)
                continue
            # other non-serializable types -> string
            try:
                json.dumps(v)
            except TypeError:
                props[k] = str(v)
    return fc

# Run loader and sanitize
geojson = load_geojson()
geojson = sanitize_props_in_geojson(geojson)
features = geojson.get('features', [])
if not features:
    raise RuntimeError('No features found in GeoJSON')

# Detect height property
height_candidates = ['b3_h_max', 'height', 'hoogte', 'building:height', 'roofHeight', 'z', 'maxh', 'hoogte_m', 'aantal_verdiepingen', 'floors']
first_props = features[0].get('properties', {})
height_col = next((c for c in height_candidates if c in first_props), None)
if height_col is None:
    # Add a default finite height if none is present
    for feat in features:
        feat.setdefault('properties', {})
        feat['properties']['height'] = 8.0
    height_col = 'height'
else:
    # ensure numeric finite heights
    for feat in features:
        props = feat.setdefault('properties', {})
        try:
            h = float(props.get(height_col, 8.0))
            if not math.isfinite(h) or h <= 0:
                h = 8.0
        except Exception:
            h = 8.0
        props[height_col] = h

# Compute center (mean centroid)
from shapely.geometry import shape
centroids = []
for feat in features:
    try:
        s = shape(feat['geometry'])
        c = s.centroid
        if math.isfinite(c.x) and math.isfinite(c.y):
            centroids.append((c.y, c.x))  # lat, lon
    except Exception:
        continue
if centroids:
    center_lat = sum(c[0] for c in centroids) / len(centroids)
    center_lon = sum(c[1] for c in centroids) / len(centroids)
else:
    # bounding box fallback
    # features may be A LOT; use first valid geometry
    from shapely.ops import unary_union
    geoms = [shape(f['geometry']) for f in features if f.get('geometry')]
    if not geoms:
        raise RuntimeError('No valid geometries to compute center')
    u = unary_union(geoms[:5000]) if len(geoms) > 5000 else unary_union(geoms)
    bbox = u.bounds
    center_lon = (bbox[0] + bbox[2]) / 2
    center_lat = (bbox[1] + bbox[3]) / 2

# Create an adaptive small sample for fast rendering in the notebook
SAMPLE_N = min(200, max(10, len(features)//10))
sample = {'type': 'FeatureCollection', 'features': features[:SAMPLE_N]}

# Print small debug info
print(f"Loaded features: {len(features)}, sample size: {len(sample['features'])}, height_col: {height_col}")

# Pydeck layer
layer = pdk.Layer(
    'GeoJsonLayer',
    sample,
    extruded=True,
    wireframe=False,
    get_fill_color=[160, 160, 220],
    get_elevation=f"properties.{height_col}",
    elevation_scale=1,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=center_lat, longitude=center_lon, zoom=15, pitch=60)

deck = pdk.Deck(layers=[layer], initial_view_state=view_state, map_style='https://demotiles.maplibre.org/style.json')

out = Path('buildings_pydeck.html')
deck.to_html(out.as_posix(), open_browser=False, notebook_display=False)
print(f"Saved pydeck HTML sample to {out.resolve()} (features={len(sample['features'])})")

# Inject MapLibre CSS into generated HTML if missing (fixes the Mapbox GL CSS warning)
html_txt = out.read_text(encoding='utf-8')
if 'maplibre-gl.css' not in html_txt and 'mapbox-gl.css' not in html_txt:
    css_link = '<link href="https://unpkg.com/maplibre-gl@2.4.0/dist/maplibre-gl.css" rel="stylesheet" />\n'
    html_txt = html_txt.replace('<head>', '<head>\n  ' + css_link)
    out.write_text(html_txt, encoding='utf-8')
    print('Injected MapLibre CSS into HTML to fix missing CSS warning')

# Add an info overlay to the HTML so user sees feature count even if map misrenders
count = len(sample['features'])
overlay_html = '<div id="pydeck-info" style="position:fixed;left:8px;top:8px;z-index:9999;background:rgba(255,255,255,0.9);padding:6px;border-radius:4px;font-size:12px">Features: ' + str(count) + '</div>'
html_txt = out.read_text(encoding='utf-8')
if '</body>' in html_txt and 'pydeck-info' not in html_txt:
    html_txt = html_txt.replace('</body>', overlay_html + '\n</body>')
    out.write_text(html_txt, encoding='utf-8')

try:
    webbrowser.open(out.as_uri())
except Exception:
    pass

# If running in Jupyter, display inline
try:
    from IPython.display import display
    display(deck)
except Exception:
    pass

Loaded features: 253762, sample size: 200, height_col: b3_h_max
Saved pydeck HTML sample to D:\Master\Q2\sci Programming for Geospatial\Project 1\Git clone\ITC-Solar-Panel-Suitability-Mapping\notebooks\buildings_pydeck.html (features=200)
Injected MapLibre CSS into HTML to fix missing CSS warning


{
  "initialViewState": {
    "latitude": 52.3527730730555,
    "longitude": 4.895094974337355,
    "pitch": 60,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "autoHighlight": true,
      "data": {
        "features": [
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    [
                      4.824343424294198,
                      52.27811905252319
                    ],
                    [
                      4.82433951693713,
                      52.27812015740987
                    ],
                    [
                      4.824340980746713,
                      52.27812213003688
                    ],
                    [
                      4.824406448847299,
                      52.27810383933808
                    ],
                    [
                      4.82433184879175,
                      52.27810378939151
                    ],
                    [
                      4.824343424294198,
                      52.27811905252319
                    ]
                  ]
                ],
                [
                  [
                    [
                      4.824223504592445,
                      52.27812381764827
                    ],
                    [
                      4.824237519555367,
                      52.27811995217305
                    ],
                    [
                      4.824239094088581,
                      52.27812220665043
                    ],
                    [
                      4.824303338179843,
                      52.27810377029053
                    ],
                    [
                      4.824208546689686,
                      52.2781037067345
                    ],
                    [
                      4.824223504592445,
                      52.27812381764827
                    ]
                  ]
                ]
              ],
              "type": "MultiPolygon"
            },
            "id": "0",
            "properties": {
              "b3_bag_bag_overlap": 0.0,
              "b3_bouwlagen": 3.0,
              "b3_dak_type": "slanted",
              "b3_dd_id": 0,
              "b3_extrusie": "standard",
              "b3_h_50p": 2.63047456741333,
              "b3_h_70p": 3.45675730705261,
              "b3_h_maaiveld": -3.76799988746643,
              "b3_h_max": 5.74852085113525,
              "b3_h_min": -1.04296052455902,
              "b3_h_nok": 5.64631652832031,
              "b3_is_glas_dak": false,
              "b3_kas_warenhuis": false,
              "b3_kwaliteitsindicator": true,
              "b3_mutatie_ahn3_ahn4": false,
              "b3_mutatie_ahn4_ahn5": false,
              "b3_n_nok": 1.0,
              "b3_n_vlakken": 7.0,
              "b3_nodata_fractie_ahn3": 0.0,
              "b3_nodata_fractie_ahn4": 0.00997506268322468,
              "b3_nodata_fractie_ahn5": 0.00249376567080617,
              "b3_nodata_radius_ahn3": 0.0,
              "b3_nodata_radius_ahn4": 0.0,
              "b3_nodata_radius_ahn5": 0.0,
              "b3_opp_buitenmuur": 179.279998779297,
              "b3_opp_dak_plat": 23.3099994659424,
              "b3_opp_dak_schuin": 98.2600021362305,
              "b3_opp_grond": 100.529998779297,
              "b3_opp_scheidingsmuur": 100.410003662109,
              "b3_pand_deel_id": 0,
              "b3_puntdichtheid_ahn3": 18.2543640136719,
              "b3_puntdichtheid_ahn4": 27.0730476379395,
              "b3_puntdichtheid_ahn5": 15.2600002288818,
              "b3_pw_bron": "ahn5",
              "b3_pw_datum": 2023,
              "b3_pw_onvoldoende": false,
              "b3_pw_selectie_reden": "PREFERRED_AND_LATEST",
              "b3_rmse_lod12": 2.34132957458496,
              "b3_rmse_lod13": 1.11082077026367,
              "b3_rmse_lod22": 0.296161502599716,
              "b3_t_run": 70,
              "b3_val3dity_lod12": [],


## 1. Load Processed Data

In [33]:
# TODO: Load processed building data
# Create a map of Mount Fuji with custom terrain source


## 2. Interactive Map with Color-Coded Suitability

In [ ]:
# TODO: Create interactive color-coded map

## 3. Pairwise Analysis Plots

In [ ]:
# TODO: Create pairwise scatter plots

## 4. Statistical Summary

In [ ]:
# TODO: Create correlation heatmap

## 5. Correlation Matrix

In [ ]:
# TODO: Visualize top priority buildings

## 6. Top 10 Priority Buildings

In [ ]:
# TODO: Export visualizations

## 7. Export Results

In [ ]:
# TODO: Create summary report

## Summary

### Generated Outputs:

1. **Interactive Map** - `outputs/maps/suitability_map.html`
   - Color-coded buildings by suitability
   - Click for detailed building information

2. **Pairwise Analysis** - `outputs/figures/pairwise_analysis.png`
   - Roof area vs. energy potential
   - Orientation vs. suitability
   - Shading factor effects

3. **Correlation Matrix** - `outputs/figures/correlation_matrix.png`
   - Factor relationships

4. **Top 10 Buildings** - `outputs/figures/top_10_buildings.png`
   - Priority ranking visualization

5. **CSV Report** - `outputs/reports/top_10_priority_buildings.csv`
   - Priority list for implementation

### Use in Technical Report:
- Include figures in Results section
- Reference interactive map
- Use statistics for Discussion